In [31]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD,Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')


import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
gpu_options = tf.GPUOptions(allow_growth=True)
config=tf.ConfigProto(gpu_options=gpu_options)

config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))



In [14]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [15]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [16]:
X_train = np.asarray([np.average(i, axis = 0) for i in X_train])
X_test = np.asarray([np.average(i, axis = 0) for i in X_test])

X_train = np.reshape(X_train, (-1, 1,32,32))
X_test = np.reshape(X_test, (-1, 1,32,32))


In [18]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [36]:
# Create the model
model = Sequential()
model.add(Conv2D(8, (10, 10), input_shape=(1,32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 50
lrate = 0.001
decay = lrate/epochs
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 8, 32, 32)         808       
_________________________________________________________________
flatten_9 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 2,100,786
Trainable params: 2,100,786
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, verbose = 1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 8s 163us/step - loss: 1.7410 - acc: 0.3768 - val_loss: 1.5875 - val_acc: 0.4334
Epoch 2/50
50000/50000 [==============================] - 8s 153us/step - loss: 1.5365 - acc: 0.4543 - val_loss: 1.5028 - val_acc: 0.4627
Epoch 3/50
50000/50000 [==============================] - 7s 147us/step - loss: 1.4203 - acc: 0.4944 - val_loss: 1.4667 - val_acc: 0.4814
Epoch 4/50
50000/50000 [==============================] - 7s 145us/step - loss: 1.3399 - acc: 0.5269 - val_loss: 1.4443 - val_acc: 0.4909
Epoch 5/50
50000/50000 [==============================] - 7s 142us/step - loss: 1.2846 - acc: 0.5448 - val_loss: 1.3910 - val_acc: 0.5104
Epoch 6/50
50000/50000 [==============================] - 7s 143us/step - loss: 1.2484 - acc: 0.5583 - val_loss: 1.4023 - val_acc: 0.5073
Epoch 7/50
50000/50000 [==============================] - 7s 143us/step - loss: 1.2151 - acc: 0.5722 - val_loss: 1.3779 - 